In [2]:
from scipy import optimize
import numpy as np
% matplotlib inline

### Deterministic Model

### Two Stage Stochastic Programming Model

### RO SP FR

### Mean Markovitz Model

In [6]:
def risk_const(x, params, risk_param, profit_lb):
    """
        Risk constraint for Mean-Markovitz model.
        
        Parameters
        ----------
        x : {np.array}
            array of decision variables
            
        params : {dict}
            dictionary of parameters used for constraint generation
        
        risk_param : {float}
            custom risk parameter for variation penalty
        
        profit_lb : {float}
            lower bound for profit (t)
            
        Parameter Dictionary
        --------------------
        sales_mean : {np.array}
            mean coefficients for sales variable (selling price)
        
        sales_std : {np.array}
            standard deviation for sales variables
            
        purchase_coeff : {np.array}
            purchase cost
        
        inventory_coeff : {np.array}
            positive inventory returns
        
        operation_coeff : {np.array}
            operation costs
    """
    # segment decision variables
    s = x[:2]
    p = x[2:4]
    I = x[4:8]
    R = x[8:]
    
    mu_q = params["sales_mean"]
    std_q = params["sales_std"]
    c = params["purchase_coeff"]
    e = params["inventory_coeff"]
    f = params["operation_coeff"]
    
    return np.dot(mu_q, y) - np.dot(c,x) + np.dot(e,I) - np.dot(f,R) - risk_param*np.sqrt(np.dot(np.square(std_q),np.square(y))) - profit_lb

In [ ]:
# profit lb
t = 10000

# risk param
theta = 0.3

# mean sales price
mean_q = np.array([90, 80])

# std sales price
std_q = np.array([27, 20])

# purchase cost
c = np.array([10, 15])

# inventory coefficients
e = np.array([10, 15, 20, 25])

# operation cost
f = np.array([30, 35])

# decision variables - s3, s4, p1, p2, I1_f, I2_f, I3_f, I4_f, R1, R2

# bounds
bounds = [(20, 40), (20, 85), (20, 100), (30, 200), (0, 20), (0, 20), (0, 20), (0, 20), (20, 100), (30, 100)]

# init vals
x = np.zeros(10)

# maximize profit
obj = lambda x: -t

params = {"sales_mean":mean_q, "sales_std": std_q, "purchase_coeff": c, "inventory_coeff": e,
          "operation_coeff": f}

# constraints
cons = (
    {"type": "ineq", "fun": risk_const, "args": (params,theta,t)},
    {"type": "eq", "fun": lambda x: x[2] - 0.4*x[8] - x[4]},
    {"type": "eq", "fun": lambda x: x[3] - 0.6*x[8] - x[9] - x[5]},
    {"type": "eq", "fun": lambda x: 0.5*x[8] - x[6] - x[0]},
    {"type": "eq", "fun": lambda x: 0.6*x[9] - x[7] - x[1]}
)

res = optimize.minimize(obj, x, bounds=bounds, constraints=cons)